# Building a Stock Analysis Application

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

__The notebook here has the mere purpose of describing and explaining the code we're going to use, not of actually run the code.__

In [2]:
import pandas as pd
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
llm_config = {
    "model": "gpt-4o",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


## Application: Financial Report

We are now going to write a complex streamlit app that will leverage agents to accomplish a specific task. We will later deploy this app on Google Cloud so that it gets a permanent link you can share with others to let them test your work.

In [5]:
import autogen

## Goals of our App

For this final example, we are going to create an app that re-uses some of the features we previously explored together. This app will do the following *<span style="color:blue">(description and goals are from the Udemy course [AI Agents: Building Teams of LLM Agents that Work For You ](https://www.udemy.com/course/ai-agents-building-teams-of-llm-agents-that-work-for-you/) by Mohsen Hassan and Ilyass Tabiai, PhD.)</span>*:

* Takes as an input asset tickers
* Downloads asset price data, analyzes them (correlations, returns, ...)
* Retrieve or compute the following ratios:
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
* Analyze the correlation between the stocks
* Plot their normalized prices for comparison
* Downloads the latest news headlines about these assets and then analyzes and summarizes these news headlines
* Writes a financial report about these assets based on all this information
    * **Legal reviewer**: Check that the content is legally compliant (in case we share it with others)
    * **Text/Data alignment reviewer**: Check that the text content is aligned with the text written in the report
    * **Consistency reviewer**: Check that the conclusion are consistent throughout the text and no contradictions are present
    * **Completion reviewer**: Check that the report contains all the elements requested
* Refines the report based on previous criticisms
* Saves the final report to a mardown file with a normalized price chart


## Defining our Agents

Our app will rely on agents to accomplish its goal, in this section, we will present the structure it will follow.

### 1. Financial & Research Task

The first task will be to get the financial data about the chosen assets and compute some of their performance metrics, the second task will be to investigate reasons for this performance based on news headlines. Let's first provide a prompt for these tasks. Note that this time we will provide both of these prompts in a list for convenience:

In [7]:
from datetime import datetime
date_str = datetime.now().strftime("%Y-%m-%d")

stocks = "AMD, NVDA"

financial_tasks = [
    f"""Today is the {date_str}. 
    What are the current stock prices of {stocks}, and how is the performance over the past 6 months in terms of percentage change? 
    Start by retrieving the full name of each stock and use it for all future requests.
    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * Analyze the correlation between the stocks
    Do not use a solution that requires an API key.
    If some of the data does not makes sense, such as a price of 0, change the query and re-try.""",

    """Investigate possible reasons of the stock performance leveraging market news headlines from Bing News or Google Search. Retrieve news headlines using python and return them. Use the full name stocks to retrieve headlines. Retrieve at least 10 headlines per stock. Do not use a solution that requires an API key.""",
]

In [9]:
financial_tasks # important to note: Do not use a solution that requires an API key

['Today is the 2024-10-08. \n    What are the current stock prices of AMD, NVDA, and how is the performance over the past 6 months in terms of percentage change? \n    Start by retrieving the full name of each stock and use it for all future requests.\n    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: \n    * P/E ratio\n    * Forward P/E\n    * Dividends\n    * Price to book\n    * Debt/Eq\n    * ROE\n    * Analyze the correlation between the stocks\n    Do not use a solution that requires an API key.\n    If some of the data does not makes sense, such as a price of 0, change the query and re-try.',
 'Investigate possible reasons of the stock performance leveraging market news headlines from Bing News or Google Search. Retrieve news headlines using python and return them. Use the full name stocks to retrieve headlines. Retrieve at least 10 headlines per stock. Do not use a solution t

We now need agents to perform these taks. We are going to define two agents to do so, a Financial agent, for the first task, and a Research agent, for the second task. Both agents will have to provide code that can perform each task, so these will be Assistant agents with LLM capability:

In [6]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

[autogen.oai.client: 10-08 11:22:17] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-08 11:22:17] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


At the end of this task, we will have the price data, performance metrics and a chart that shows normalized price evolution fot the specified assets. 

### 2. Writing task

The second taks will be to write a first version of a report based on the data provided by the previous agents. Let's define the task:

In [11]:
writing_tasks = [
        """Develop an engaging financial report using all information provided, include the normalized_prices.png figure,
        and other figures if provided.
        Mainly rely on the information provided. 
        Create a table comparing all the fundamental ratios and data.
        Provide comments and description of all the fundamental ratios and data.
        Compare the stocks, consider their correlation and risks, provide a comparative analysis of the stocks.
        Provide a summary of the recent news about each stock. 
        Ensure that you comment and summarize the news headlines for each stock, provide a comprehensive analysis of the news.
        Provide connections between the news headlines provided and the fundamental ratios.
        Provide an analysis of possible future scenarios. 
        """]

In [13]:
writing_tasks

['Develop an engaging financial report using all information provided, include the normalized_prices.png figure,\n        and other figures if provided.\n        Mainly rely on the information provided. \n        Create a table comparing all the fundamental ratios and data.\n        Provide comments and description of all the fundamental ratios and data.\n        Compare the stocks, consider their correlation and risks, provide a comparative analysis of the stocks.\n        Provide a summary of the recent news about each stock. \n        Ensure that you comment and summarize the news headlines for each stock, provide a comprehensive analysis of the news.\n        Provide connections between the news headlines provided and the fundamental ratios.\n        Provide an analysis of possible future scenarios. \n        ']

In [14]:
# let's define an agent to work on the writing task
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging finance reports.
        You transform complex concepts into compelling narratives. 
        Include all metrics provided to you as context in your analysis.
        Only answer with the financial report written in markdown directly, do not include a markdown language block indicator.
        Only return your final work without additional comments.
        """,
)
writer

[autogen.oai.client: 10-08 13:00:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### 3. Refining the blogpost

To refine the blogpost based on SEO, legal aspects and ethical aspects, we are going to use a nested chat that will be triggered when the Writer contacts the Critic. The Critic will trigger a nested chat with agents designed to optimize each aspect. A final agent in this nested chat, the Meta Reviewer, will summarize and assemble all reviews and send them back to the Critic, who'll transfer them to the Writer agent to prepare a refined version. Let's define all these agents: